In [1]:
import json
from typing import Optional, Dict, Any, List
import utils as util

from scienceworld import ScienceWorldEnv
from textworld import EnvInfos

# from alfworld.agents.environment import get_environment
# import alfworld.agents.modules.generic as generic

# Textworld

### Wrapper

In [2]:
cooking_wrapper = util.Textworld_Cooking_Wrapper_Env()

# Generate new game and load env in
cooking_wrapper.generate_new_game()
cooking_wrapper.load_env()

while not cooking_wrapper.game_state.done:
    cooking_wrapper.step_env()
    if cooking_wrapper.game_state.moves == 10:
        print(cooking_wrapper.get_env_state(full_state=True))
    # else:
        # print(cooking_wrapper.get_env_state(full_state=False))

< Full Game State >
Move 10 | Score = 6/15
Actionable Verbs: ['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'examine', 'go', 'insert', 'inventory', 'lock', 'look', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
Objective: You are hungry! Let's cook a delicious meal. Check the cookbook in the kitchen for the recipe. Once done, enjoy your meal!
Location: Kitchen
Inventory: You are carrying: a red onion, a raw pork chop, a fried banana, a fried block of cheese and a roasted orange bell pepper.
Current Observation: You arrive in a kitchen. A typical kind of place.

You can see a fridge. Hmmm... what else, what else? The fridge contains a red apple, a white onion and a yellow bell pepper. You can make out a closed oven. You can make out a table. On the table you can make out a cookbook and a knife. You see a counter. The counter is vast. On the counter you see a raw purple potato and a raw red potato. Suddenly, you bump your head on the ceiling, but it's not such a bad bump

### Playable Textworld

In [4]:
test_playable_tw = False

if test_playable_tw:
    play_textworld = False
    tw_filename = "custom_cooking"
    tw_folder = "tw_games"

    # Instantiate our game
    util.instantiate_textworld_cooking_game(
        filename=tw_filename,
        folder=tw_folder,
        recipe=5, take=3, go=1,
        open_=True, cook=True, cut=True, drop=True,
        split="train"
    )
    info_requests = EnvInfos(
        # admissible_commands=True,
        description=True,
        inventory=True,
        location=True,
        objective=True,
        verbs=True
    )

    # Load our game and print initial obs
    env, env_info = util.load_textworld_cooking_game(filename=tw_filename, request_infos=info_requests, folder=tw_folder, max_episode_steps=100)
    obs, env_state = env.reset()
    print(env_state)
    print(f"Initial Observation:\n{util.clean_obs_cooking(obs)}\n##################\n")
    score, moves, done = 0, 0, False

    # Main play loop
    while not done:
        if play_textworld:
            command = input("> ")
        else:
            command = env_info['walkthrough'][moves]
        if command == "stop":
            break
        obs, score, done, env_state = env.step(command)
        print(f"Action:\n> {command}\n\nObservation:{util.clean_obs_cooking(obs)}\n\nScore: {score}\n#################\n")
        moves += 1

    # Final print
    print(f"Moves: {moves}; Score: {score} / {env_info['max_score']}; Done: {done}")
    print(f"Gold Path:\n{env_info['walkthrough']}")
    env.close()

# Science World

In [2]:
# Pretty-print helpers (optional)
def hr(title: Optional[str] = None):
    print("\n" + "-"*90)
    if title:
        print(title)
        print("-"*90)

def pkey(d: Dict[str, Any], key: str, label: Optional[str] = None):
    if key in d:
        print(f"{label or key}: {d[key]}")

# Environment bootstrap (works with or without a custom JAR)
def init_env(jar_path: Optional[str] = None, env_step_limit: int = 100) -> ScienceWorldEnv:
    """
    If jar_path is None or "", ScienceWorldEnv uses the built-in jar.
    """
    env = ScienceWorldEnv("", jar_path or "", envStepLimit=env_step_limit)
    return env

def list_tasks(env: ScienceWorldEnv) -> List[str]:
    tasks = env.get_task_names()
    for i, t in enumerate(tasks):
        print(f"{i:2d}: {t}")
    return tasks

def load_task(
    env: ScienceWorldEnv,
    task_name: str,
    var_num: int = 0,
    simplifications: Optional[str] = "easy",
    generate_gold_path: bool = True
):
    """
    simplifications: e.g., "easy" or a comma-joined string like "teleportAction,openDoors,selfWateringFlowerPots,noElectricalAction"
    """
    env.load(task_name, var_num, simplifications or "", generateGoldPath=generate_gold_path)
    obs, info = env.reset()
    return obs, info

In [3]:
# Use default envs defined in science world
env = init_env(jar_path=None, env_step_limit=100)
tasks = list_tasks(env)

 0: boil
 1: change-the-state-of-matter-of
 2: chemistry-mix
 3: chemistry-mix-paint-secondary-color
 4: chemistry-mix-paint-tertiary-color
 5: find-animal
 6: find-living-thing
 7: find-non-living-thing
 8: find-plant
 9: freeze
10: grow-fruit
11: grow-plant
12: identify-life-stages-1
13: identify-life-stages-2
14: inclined-plane-determine-angle
15: inclined-plane-friction-named-surfaces
16: inclined-plane-friction-unnamed-surfaces
17: lifespan-longest-lived
18: lifespan-longest-lived-then-shortest-lived
19: lifespan-shortest-lived
20: measure-melting-point-known-substance
21: measure-melting-point-unknown-substance
22: melt
23: mendelian-genetics-known-plant
24: mendelian-genetics-unknown-plant
25: power-component
26: power-component-renewable-vs-nonrenewable-energy
27: test-conductivity
28: test-conductivity-of-unknown-substances
29: use-thermometer


In [4]:
# # Choose task to load / play around with
# task_idx = 6
# task_name = tasks[task_idx]
# print(f"Loading task #{task_idx}: {task_name}.")

# # Load task and print initial observation
# obs, info = load_task(env, task_name=task_name, var_num=1, simplifications="easy", generate_gold_path=True)
# print(f"Task: {env.get_task_description()}")
# print(f"Goal Progress: {env.get_goal_progress()}")
# gold_actions = env.get_gold_action_sequence()
# print(gold_actions)

# for idx, a in enumerate(gold_actions):
#     print(f"Step {idx+1}:\n{obs}")
#     print(f"Action: {a}")
#     obs, reward, isCompleted, infos = env.step(a)
#     print(f"Reward: {reward:.2f} | Score: {infos['score']:.2f} |  Completed? {isCompleted}\n\n")

In [5]:
play_scienceworld = False

if play_scienceworld:
    # Load task    
    task_idx = 8
    task_name = tasks[task_idx]
    print(f"Loading task #{task_idx}: {task_name}.")
    obs, info = load_task(env, task_name=task_name, var_num=1, simplifications="easy", generate_gold_path=True)
    print(f"Task: {env.get_task_description()}")
    score, moves, done = 0, 0, False
    
    # Render loop
    reward = 0
    while not done:
        command = input("> ")
        obs, score, done, infos = env.step(command)
        reward += score
        print(f"Observation:\n{obs}\n")
        print(f"Score: {score}; Total Reward: {reward}; Done: {done}\n####################\n\n")
        moves += 1

    print(f"Moves: {moves}; Reward: {reward}\n")
    print(f"Gold Path:\n{env.get_gold_action_sequence()}")
    print(f"Goal Progress:\n{env.get_goal_progress()}")
    env.reset()

# Text World

Global seed: 3351
Game generated: /home/lucas/code/seq-decision/sequential-decision-processors/data/textworld/raw/tw_games/custom_cooking.z8
{'objective': "You are hungry! Let's cook a delicious meal. Check the cookbook in the kitchen for the recipe. Once done, enjoy your meal!", 'verbs': ['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'examine', 'go', 'insert', 'inventory', 'lock', 'look', 'open', 'prepare', 'put', 'slice', 'take', 'unlock'], 'location': None, 'inventory': 'You are carrying: a red apple, a banana and a raw red potato.', 'description': "-= Kitchen =-\nThis is going to sound unbelievable, but you've just entered a kitchen. The room seems oddly familiar, as though it were only superficially different from the other rooms in the building.\n\nYou scan the room for a fridge, and you find a fridge. Suddenly, you bump your head on the ceiling, but it's not such a bad bump that it's going to prevent you from looking at objects and even things. You hear a noise behind